#### [일변량 데이터기반 회귀 모델]
- 주제 : 학습 시간에 따른 수능 점수 예측 서비스
- 데이터 : study_score_easy.csv
- 구성 : 피쳐 + 타겟
- 학습 : 지도학습 + 회귀
- 구현 : 인공신경망


[1] 모듈로딩 및 데이터 준비<hr>

In [74]:
# %pip install visdom

In [75]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchinfo import summary

In [76]:
data_file = '../Data/study_score_easy.csv'

## 데이터 로딩
dataDF = pd.read_csv(data_file)
dataDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   hours   1000 non-null   float64
 1   score   1000 non-null   float64
dtypes: float64(2)
memory usage: 15.8 KB


In [77]:
## [1-3] 데이터 -> 텐서 변환
featureDF = dataDF[dataDF.columns[0:1]]
xTS = torch.tensor(featureDF.values, dtype=torch.float32)

targetDF = dataDF[dataDF.columns[1:]]
yTS = torch.tensor(targetDF.values,  dtype=torch.float32)

print("xTS : ", xTS.shape,xTS.dim, "\nyTS : ", yTS.shape, yTS.dim )

xTS :  torch.Size([1000, 1]) <built-in method dim of Tensor object at 0x0000022FBD3F1E40> 
yTS :  torch.Size([1000, 1]) <built-in method dim of Tensor object at 0x0000022FBD3F2E30>


[2] ANN 모델 설계 <hr>

In [78]:
## ==========================================================
##          입력수      퍼셉트론수/출력수       AF
## ==========================================================
## 입력층     1              1               ★Pytorch에는 입력층(클래스) X, 입력 텐서를 입력층으로 간주
## 은닉층     1              10              ReLU
## 은닉층     10             15              ReLU
## 출력층     15             10              X(회귀라서)
## ==========================================================
## 클래스이름 : ScoreModel
## 부모클래스 : nn.Module
## 오버라이딩 : __init__(self)  : 층 구성 요소 인스턴스 생성
##            forward(self, x) : 순전파 진행 메서드, ★ x:입력층
## ==========================================================
class ScoreModel(nn.Module) :
    ## 층 구성 인스턴스 생성 메서드
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.hd1_layer= nn.Linear(1, 10)
        self.hd2_layer= nn.Linear(10, 15)
        self.out_layer= nn.Linear(15, 1)

    ## 순전파 진행 메서드 
    def forward(self, x) :
        ## input -> hd1 : 1 -> 10
        out = self.hd1_layer(x)
        out = F.relu(out)
        
        ## hd1 -> hd2 : 10 -> 15
        out = self.hd2_layer(out)
        out = F.relu(out)
        
        ## hd2 -> out : 15 -> 1
        ## 회귀모델로 AF없음
        out = self.out_layer(out)
        
        return out

In [79]:
## 모델 객체 생성
model = ScoreModel()
summary(model, input_size=(1,1))

Layer (type:depth-idx)                   Output Shape              Param #
ScoreModel                               [1, 1]                    --
├─Linear: 1-1                            [1, 10]                   20
├─Linear: 1-2                            [1, 15]                   165
├─Linear: 1-3                            [1, 1]                    16
Total params: 201
Trainable params: 201
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

[3] 학습 준비 <hr>

In [80]:
## [3-1] 학습 관련 설정값들
EPOCHS      = 1000                              ## 처음 ~ 끝까지 학습 횟수
BATCH_SIZE  = 200                               ## 1번 학습할 데이터 크기
COUNT       = featureDF.shape[0] / BATCH_SIZE   ## 1에포크에 파라미터 업데이트 횟수
COUNT       

5.0

In [81]:
## [3-2] 학습 관련 인스턴스들
## -> 모델 인스턴스
model = ScoreModel()

## -> 손실 계산 객체
loss_fn = nn.MSELoss()

## -> 최적화 인스턴스
adamOpt = optim.Adam(model.parameters())

[4] 학습진행 <hr>

In [82]:
## 에포크 당 loss 저장 변수
history = []
## 학습 진행

for epoch in range(0, EPOCHS + 1) :
    total_loss = 0
    for idx in range(int(COUNT)) :
        # print(idx*200)
        # 배치크기만큼 순전파 진행 => 예측값 추출
        pre_y = model(xTS[idx*BATCH_SIZE : idx*BATCH_SIZE+200]) ## 200개씩 잘라서
        
        # 손실계산 
        loss = loss_fn(pre_y, yTS[idx*BATCH_SIZE : idx*BATCH_SIZE+200])
        # print(pre_y.dtype)   
        
        ## 역전파 + 최적화
        adamOpt.zero_grad()
        loss.backward()         ## <- 역전파 : 경사하강법으로 계산
        adamOpt.step()          ## <- 새로운 W,b 업데이트 진행
        
        ## 배치 크기 loss 누적
        total_loss += loss.item()
        
    if not epoch%50 :
        print(f'[{epoch:03}_에포크 : loss:{total_loss/COUNT:f}')
        

[000_에포크 : loss:4044.262061
[050_에포크 : loss:2321.985889
[100_에포크 : loss:23.631329
[150_에포크 : loss:12.271684
[200_에포크 : loss:6.946939
[250_에포크 : loss:3.977130
[300_에포크 : loss:2.314792
[350_에포크 : loss:1.483071
[400_에포크 : loss:1.145095
[450_에포크 : loss:1.045605
[500_에포크 : loss:1.035714
[550_에포크 : loss:1.035497
[600_에포크 : loss:1.035497
[650_에포크 : loss:1.035498
[700_에포크 : loss:1.035501
[750_에포크 : loss:1.035503
[800_에포크 : loss:1.035506
[850_에포크 : loss:1.035509
[900_에포크 : loss:1.035512
[950_에포크 : loss:1.035517
[1000_에포크 : loss:1.035521
